# QrackCircuit and Bluequbit

In [1]:
import random
import time

# How wide of an example circuit should we generate? (In qubits)
width = 32

All this next cell does is provide a function for generating random circuits, for our example that follows.

In [2]:
def random_circuit(width, sim):
    single_bit_gates = sim.h, sim.x, sim.y, sim.z, sim.s, sim.t 
    two_bit_gates = sim.cx, sim.cz
    all_bits = list(range(0, width))
    gate_count = 0
    
    for i in range(width):
        # Single bit gates
        for j in range(width):
            gate = random.choice(single_bit_gates)
            gate(j)
            
        # Fully-connected couplers:
        ###########################
        unused_bits = random.sample(all_bits, width)
        while len(unused_bits) > 1:
            b1 = unused_bits.pop()
            b2 = unused_bits.pop()
            gate = random.choice(two_bit_gates)
            gate(b1, b2)

The circuit starts in Qiskit representation. This is converted to a `QrackCircuit`, and the gates are "locally simplified" in the conversion process. The `QrackCircuit` is then converted back an (optimized) Qiskit representation, which can be directly run by `bluequbit`. (Theoretically, Qiskit Aer and PyQrack have no trouble running the "intermediate representation" in the middle step, but `qsimcirq` underlies `bluequbit` and does not handle "uniformly controlled gates.")

In [3]:
from pyqrack import QrackCircuit
from qiskit import QuantumCircuit
from qiskit.compiler.transpiler import transpile

qcirc = QuantumCircuit(width)
random_circuit(width, qcirc)
print("Gate count before optimization: ", sum(qcirc.count_ops().values()))

qcirc = QrackCircuit.in_from_qiskit_circuit(qcirc)
qcirc.out_to_file("bq_circuit.qc")
qcirc = QrackCircuit.file_to_qiskit_circuit("bq_circuit.qc")
print("Gate count in intermediate representation: ", sum(qcirc.count_ops().values()))
                                            
basis_gates = ["u", "cx", "rz"]
qcirc = transpile(qcirc, basis_gates=basis_gates, optimization_level=3)
print("Gate count after optimization: ", sum(qcirc.count_ops().values()))

qcirc.measure_all()

Gate count before optimization:  1536
Gate count in intermediate representation:  626
Gate count after optimization:  1507


In [4]:
import bluequbit

start = time.perf_counter()
bq = bluequbit.init("YOUR_TOKEN_HERE")
result = bq.run(qcirc, device='cpu', shots=1024)
print(result.get_counts())
shots = 0
for key, value, in result.get_counts().items():
    shots += value
print(shots)
end = time.perf_counter()
print("Time: ", end - start)

BQ-PYTHON-SDK - WARNING - Development version 0.5.0b1 of BlueQubit Python SDK is being used
BQ-PYTHON-SDK - INFO - Submitted: Job ID: dfE6WlWVXy2Cwlkm, device: cpu, run status: RUNNING, created on: 2023-09-29 13:29:39 UTC, estimated runtime: 2990000 ms, estimated cost: $0.00, num qubits: 32


{'00000000010111010000011010001000': 1, '00000000101111010000100110011111': 1, '00000000110010001000011000101110': 1, '00000001011001010000100000011111': 1, '00000001101001110011110011010101': 1, '00000001101100110110100000111100': 1, '00000001111000110110101110001001': 1, '00000010001000000001110111010111': 1, '00000010011000111001001110000110': 1, '00000010110100100111101111111100': 1, '00000011010001010011111001110100': 1, '00000011011101111001110010100110': 1, '00000011100000011101010111101010': 1, '00000011100010110100100101000010': 1, '00000011111100101000111011101101': 1, '00000100000100011101010100101111': 1, '00000100000110100011001100101110': 1, '00000100010100111111111000110110': 1, '00000100011000101000010101011010': 1, '00000100011101011100001011110100': 1, '00000100011110000000110000101000': 1, '00000100011111010001111100100110': 1, '00000101000010010000010001111110': 1, '00000101000010110101100111010111': 1, '00000101001111111100101110010000': 1, '00000101110001101000101

**(Happy Qracking! You rock!)**